In [ ]:
!pip install neo4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install psycopg2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from neo4j import GraphDatabase
import pandas as pd
import psycopg2

In [ ]:
!curl ipecho.net/plain

35.230.168.188

In [ ]:
# Define Neo4j connection details
neo4j_uri = "neo4j+s://819248de.databases.neo4j.io"
neo4j_user = "neo4j"
neo4j_password = "Ixzg5zFUEnbJ2p-eZ8Lt199u2QyutizkJyjWivjHlhc"
driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

In [ ]:
# Define Postgres connection details
Postgres_host = "34.170.37.217"
Postgres_database = "Neo4j_db"
Postgres_user = "student2"
Postgres_password = '$YBO>"X@D+xNFIKv'

In [ ]:
# Extract data from Neo4j
def extract_data():
    with driver.session() as session:
        query = "MATCH (c:Customer)-[r:SUBSCRIBES]->(s:Service) RETURN c.id AS customer_id, s.id AS service_id, r.subscription_id AS subscription_id, r.start_date AS start_date, r.end_date AS end_date, r.price AS price"
        result = session.run(query)
        data = [record for record in result]
        return data

In [ ]:
# Data transformation
def transform_data(data):
    df = pd.DataFrame(data)
    print(df.columns)  # print the column names
    df['start_date'] = pd.to_datetime(df['start_date'])
    df['end_date'] = pd.to_datetime(df['end_date'])
    df = df.dropna()
    return df

In [ ]:
# Load data into Postgres
def load_data(df):
    conn = psycopg2.connect(
    host="34.170.37.217",
    database="Neo4j_db",
    user="student2",
    password="$YBO>\"X@D+xNFIKv"
)
    cursor = conn.cursor()
    create_table_query = '''
    CREATE TABLE subscriptions (
        customer_id INTEGER,
        subscription_id INTEGER,
        service_id INTEGER,
        start_date DATE,
        end_date DATE,
        price FLOAT
    )
    '''
    cursor.execute(create_table_query)
    conn.commit()
    for index, row in df.iterrows():
        insert_query = f"INSERT INTO subscriptions (customer_id, subscription_id, service_id, start_date, end_date, price) VALUES ({row['customer_id']}, {row['subscription_id']}, {row['service_id']}, '{row['start_date'].date()}', '{row['end_date'].date()}', {row['price']})"
        cursor.execute(insert_query)
        conn.commit()

# Call main function
def main():
  if __name__ == "__main__":
    main()

**Data Pipeline Setup and Execution Guide**

This data pipeline is designed to extract data from a Neo4j database, transform it, and load it into a Postgres database. The code is written in Python and uses the neo4j and psycopg2 libraries for database connectivity and pandas for data transformation. I have explained the purpose of each code function through comments.

**Data Schema**

The data schema used in this pipeline consists of a Customer node and a Service node in a Neo4j database, with a SUBSCRIBERS relationship between them. Each SUBSCRIBERS relationship has a subscription_id, start_date, end_date, and price property.

**The data is transformed into a Pandas DataFrame with columns:**

customer_id: integer representing the id of the customer
service_id: integer representing the id of the service
subscription_id: integer representing the id of the subscription
start_date: date representing the start date of the subscription
end_date: date representing the end date of the subscription
price: float representing the price of the subscription
The transformed data is then loaded into a Postgres database with a table named subscriptions with the same columns as in the transformed DataFrame.

**Transformations**

The following transformations are performed on the extracted data:

Convert the start_date and end_date columns to datetime format
Drop any rows with missing values.

**Running the Data Pipeline**

Run the main() function to execute the data pipeline.The transformed data should now be loaded into the subscriptions table in the Postgres database.